# Pulsar Stars 

# Introduction

Pulsar stars are a rare type of neutron star that emit radio waves through their rapid and consistent rotation, generating periodic radio emissions. Astronomers detect potential pulsar stars through these radio emmisions and label them as 'candidates' based on the average features observed in their rotational patterns. They are of considerable scientific interest as probes of space-time, the inter-stellar medium, and states of matter, and thus are important to physics and astronomy research (Skelly, 2017). Almost 2000 pulsar stars have been discovered, and there are many more undiscovered pulsars. (Keith et al., 2010).

In our project we will be using the dataset HTRU2. It describes a sample of pulsar candidates collected during the High Time Resolution Universe Survey (South). The data format is CVS, the dataset is multivariate.

It includes:

1. 1,639 pulsar stars
2. 16259 RFI/noise (false candidates)

The candidates are described by 8 continuous variables:

- Mean of the integrated profile.
- Standard deviation of the integrated profile.
- Excess kurtosis of the integrated profile.
- Skewness of the integrated profile.
- Mean of the DM-SNR curve.
- Standard deviation of the DM-SNR curve.
- Excess kurtosis of the DM-SNR curve.
- Skewness of the DM-SNR curve.
- Class variable: 0 (non-pulars) or 1 (pulsars).

The question we aim to address is:

Given a candidate pulsar star’s integrated profile excess kurtosis and the standard deviation of the DM-SNR curve, can we accurately determine if the presented candidate is a pulsar star or not? not with high accuracy??ables in the dataset)

# Methods and Results

In [ ]:
# First, we need to import the neccessary libraries
library(tidyverse)
library(tidymodels)
library(cowplot)
require(gridExtra)
library(GGally)
install.packages("themis")
library(themis)
set.seed(2024) 

Warning message:
“package ‘ggplot2’ was built under R version 4.3.2”
── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.3     ✔ readr     2.1.4
✔ forcats   1.0.0     ✔ stringr   1.5.0
✔ ggplot2   3.5.0     ✔ tibble    3.2.1
✔ lubridate 1.9.2     ✔ tidyr     1.3.0
✔ purrr     1.0.2     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors
── Attaching packages ────────────────────────────────────── tidymodels 1.1.1 ──

✔ broom        1.0.5     ✔ rsample      1.2.0
✔ dials        1.2.0     ✔ tune         1.1.2
✔ infer        1.0.5     ✔ workflows    1.1.3
✔ modeldata    1.2.0     ✔ workflowsets 1.0.1
✔ parsnip      1.1.1     ✔ yardstick    1.2.0
✔ recipes      1.0.8     

── Conflicts ───────────────────────────────────────── tidymodels_co

To begin, we will read the data frame and assign names to each column:

In [ ]:
star_data <- read_csv("HTRU_2.csv", col_names = FALSE, show_col_types = FALSE) |>
    mutate(X9 = as_factor(X9))

colnames(star_data) <- c("integrated_profile_mean", "integrated_profile_std", 
                         "integrated_profile_excess_kurtosis","integrated_profile_skewness",
                         "dm_snr_curve_mean","dm_snr_curve_std",
                         "dm_snr_curve_excess_kurtosis","dm_snr_curve_skewness",
                         "is_pulsar_star")

head(star_data)

**Table 1:** Pulsar Star Data

By using the `group_by` and `summarize` functions, we noticed the data is imbalanced, with 16259 negative results and 1639 positive results.

In [ ]:
star_summary <- star_data |>
    group_by(is_pulsar_star) |> 
    summarize(count = n())

star_summary

**Table 2:** Unbalanced Pulsar Star Data

To balance the data, we will be using the function `step_upsample` from the Themis library. First, we will split the data into 75% training and 25% testing data. Then we will scale, center, and balance the data using all the predictors. This gives us enough data to train the classifier on, as well as enough to test it on later. This means our classifier is going to be reliable. Now, we can see that the data is balanced.

In [ ]:
set.seed(4321)
star_split <- initial_split(star_data, prop = 0.75, strata = is_pulsar_star)
star_training <- training(star_split)
star_testing <- testing(star_split)

head(star_training)

star_recipe <- recipe(is_pulsar_star ~., data = star_training) |>
    step_upsample(is_pulsar_star, over_ratio = 1, skip = FALSE) |>
    step_scale(all_predictors()) |>
    step_center(all_predictors()) |>
    prep()

balanced_star <- bake(star_recipe, star_training) 
head(balanced_star)

**Table 3:** Pulsar Star Training Set

In [ ]:
# To see that the data is balanced
balanced_stars <- balanced_star |>
    group_by(is_pulsar_star) |> 
    summarize(count = n())

balanced_stars

**Table 4:** Pulsar Star Balanced Data

We can also visulize the distribution of each of the eight variables for pulsar and non-pulsar stars to get a rough picture of what the respective counts for each of our relevant variables were, we decided to create histograms to show the distributions

In [ ]:
options(repr.plot.width = 9, repr.plot.height = 10)

integrated_profile_mean_graph <- balanced_star |>
    ggplot(aes(x = integrated_profile_mean, fill = is_pulsar_star)) +
    geom_histogram(binwidth = 0.5) +
    labs(x = "Integrated Profile Mean", fill = "Pulsar Star?", title = "Count vs Integrated Profile Mean") +
    theme(text = element_text(size = 12))  

integrated_profile_std_graph <- balanced_star |>
    ggplot(aes(x = integrated_profile_std, fill = is_pulsar_star)) +
    geom_histogram(binwidth = 0.5) +
    labs(x = "Integrated Profile STD", fill = "Pulsar Star?", title = "Count vs Integrated Profile STD") +
    theme(text = element_text(size = 12))  

integrated_profile_excess_kurtosis_graph <- balanced_star |>
    ggplot(aes(x = integrated_profile_excess_kurtosis, fill = is_pulsar_star)) +
    geom_histogram(binwidth = 0.5) +
    labs(x = "Integrated Profile Excess Kurtosis", fill = "Pulsar Star?", title = "Count vs Integrated Profile Excess Kurtosis") +
    theme(text = element_text(size = 12))   

integrated_profile_skewness_graph <- balanced_star |>
    ggplot(aes(x = integrated_profile_skewness, fill = is_pulsar_star)) +
    geom_histogram(binwidth = 0.5) +
    labs(x = "Integrated Profile Skewness", fill = "Pulsar Star?", title = "Count vs Integrated Profile Skewness") +
    theme(text = element_text(size = 12))  

dm_snr_curve_mean_graph <- balanced_star |>
    ggplot(aes(x = dm_snr_curve_mean, fill = is_pulsar_star)) +
    geom_histogram(binwidth = 0.5) +
    labs(x = "DM SNR Curve Mean", fill = "Pulsar Star?", title = "Count vs DM SNR Curve Mean") +
    theme(text = element_text(size = 12))  

dm_snr_curve_std_graph <- balanced_star |>
    ggplot(aes(x = dm_snr_curve_std, fill = is_pulsar_star)) +
    geom_histogram(binwidth = 0.5) +
    labs(x = "DM SNR Curve STD", fill = "Pulsar Star?", title = "Count vs DM SNR Curve STD") +
    theme(text = element_text(size = 12))  

dm_snr_curve_excess_kurtosis_graph <- balanced_star |>
    ggplot(aes(x = dm_snr_curve_excess_kurtosis, fill = is_pulsar_star)) +
    geom_histogram(binwidth = 0.5) +
    labs(x = "DM SNR Curve Excess Kurtosis", fill = "Pulsar Star?", title = "Count vs DM SNR Curve Excess Kurtosis") +
    theme(text = element_text(size = 12))  

dm_snr_curve_skewness_graph <- balanced_star |>
    ggplot(aes(x = dm_snr_curve_skewness, fill = is_pulsar_star)) +
    geom_histogram(binwidth = 0.5) +
    labs(x = "DM SNR Curve Skewness", fill = "Pulsar Star?", title = "Count vs DM SNR Curve Skewness") +
    theme(text = element_text(size = 12))  

grid.arrange(integrated_profile_mean_graph, 
             integrated_profile_std_graph, 
             integrated_profile_excess_kurtosis_graph, 
             integrated_profile_skewness_graph, 
             dm_snr_curve_mean_graph, 
             dm_snr_curve_std_graph, 
             dm_snr_curve_excess_kurtosis_graph, 
             dm_snr_curve_skewness_graph,
             ncol = 2)

**Figure 1:** Plots of Distribution of Variable

### Building and Training Our Classifier Model 

From the figures above, we can find that some variables have distinct characteristics that may be able to determine whether or not it is a pulsar. This is because there are different trends in the distribution of pulsars that can be visualized on the graphs, such as the integrated profile excess kurtosis and the DM SNR curve std. Therefore, we are going to analyze this research question: ***Can the integrated profile excess kurtosis and DM SNR curve STD predict whether a pulsar star candidate is a pulsar star?***

To do this, we are going to use the nearest_neighbor() function for building our classifier and utilized cross-validation with tune() to determine the best K-value for accurate analysis. Scaling variables was essential to ensure equitable distance calculations among predictors. Due to errors encountered when incorporating the step_upsample function into our recipe for tuning, we opted for manual upsampling before splitting the data. Additionally, we used the vfold() function to create a 5-fold validation set, balancing computational efficiency and statistical significance. Our choice of 5 folds was driven by computational constraints, with higher fold counts proving inefficient for our large dataset. We will try to find out the best k which shows the highest accuracy, supported by a plot illustrating the peak K-value. Armed with the optimal K-value, we then will train our classification model on the train data, enabling predictions on the test data and subsequent construction of the confusion matrix.


In [ ]:
set.seed(21)
pulsar_vfold <- vfold_cv(star_training, v = 5, strata = is_pulsar_star)

stars_recipe <- recipe(is_pulsar_star ~ dm_snr_curve_std + integrated_profile_excess_kurtosis, data = balanced_star) |> 
      prep()
     

# Create KNN for tuning k
knn_tune <- nearest_neighbor(weight_func = "rectangular", neighbors = tune()) |>
      set_engine("kknn") |>
      set_mode("classification")

# create KNN Results
gridvals <- tibble(neighbors = seq(from = 1, to = 25, by = 1))

knn_predicts <-  workflow() |>
                add_recipe(stars_recipe) |>
                add_model(knn_tune) |>
                tune_grid(resamples = pulsar_vfold, grid = gridvals) |>
                collect_metrics() |>
                filter(.metric == "accuracy")
knn_predicts

In [ ]:
set.seed(21)  

options(repr.plot.width = 7, repr.plot.height = 7)

# Finding the number of neighbors that yields the highest accuracy (mean)
k_max <- knn_predicts |>
    arrange(desc(mean)) |> 
    slice(1)
k_max

# Plotting accuracy against neighbors
accuracies_vs_k  <- ggplot(knn_predicts, aes(x = neighbors, y = mean))+
       geom_point() +
       geom_line() +
       ggtitle('Accuracies vs Neighbors') +
       labs(x = "Neighbors", y = "Accuracy Estimate") +
       scale_x_continuous(breaks = seq(0, 25, 1)) + 
       theme(text = element_text(size = 16))
accuracies_vs_k

**Figure 2:**  *The figure is displaying the estimated accuracy vs the number of neighbors used for the analysis. The figure shows an increasing accuracy estimate as the number of neighbors is increased.*

From the Figure 2, We can see that there is a strong transition at k = 3, after which the data fluctuates slightly and stays flat, so we choose k = 3 as our most appropriate option for k. By knowing the appropriate K-value, we were then able to specify our classification model for our train data. Through this, we were eventually able to make predictions for the test data, which allowed us to configure our confusion matrix. We then illustrated the matrix using a four-fold plot, and found the number of observations our classifier correctly labeled as pulsar.

In [ ]:
set.seed(21)  

k_best <- 3

# Specifying model using best number of neighbors
star_spec <- nearest_neighbor(weight_func = "rectangular", neighbors = k_best) |>
  set_engine("kknn") |>
  set_mode("classification")
 
# Fitting our model to the training data
star_fit <- workflow() |>
  add_recipe(stars_recipe) |>
  add_model(star_spec) |>
  fit(data = star_training)
 
# Evaluating our model with the testing data
star_predictions <- star_fit |>
  predict(star_testing) |>
  bind_cols(star_testing) |>
  metrics(truth = is_pulsar_star, estimate = .pred_class) |>
  filter(.metric == 'accuracy')
star_predictions

star_matrix <- star_fit |>
    predict(star_testing) |>
    bind_cols(star_testing) |>
    conf_mat(truth = is_pulsar_star, estimate = .pred_class)
star_matrix

We can see from the table above that the accuracy is 97% which is very high. As we can see from this confusion matrix table, there were 4025 True Negatives and 353 True positivies. This means that at all the data points, the model predicted 4378 points correctly. With 30 False Positives and 67 False Negatives, that means that the model only made 97 wrong predictions. This is a good one to show that a k nearest neighbors model performs well in this type of dataset.

# Discussion

### Answering the predictive question

the predictive question is "Can the integrated profile excess kurtosis and DM SNR curve STD predict whether a pulsar star candidate is a pulsar star?" To answer this question, we made a classifier that determines whether a pulser star candidate is a pulsar star by integrated profile excess kurtosis and DM SNR curve STD.

According to Figure 1, we can see that the two variables, integrated profile excess kurtosis and DM SNR curve STD, are good predictors of whether a pulsar signal is real or not because our histogram shows a clear classification, from our histogram we can see that the false signals are all concentrated in the part where the integrated profile excess kurtosis is small, as the integrated profile excess kurtosis becomes larger, the pulsar signal is almost all real, and the same is true for the DM SNR curve STD, the same is true for the DM SNR curve STD, the real pulsar signals are all concentrated in the part where the DM SNR curve STD is larger.
### Expected result vs actual outcomes

We predict that the results obtained with our classifier should be of high accuracy, which should be at least greater than, 80%, in order to show that our classifier can be applied in practice, and the accuracy of the classifier is important because it involves the probability of our estimation of whether or not the detected signals come from a real pulsar or not

According to our calculations, we found that our result is 97%, 97% is the result of high accuracy, which shows that our classifier has high accuracy, and high utility, and it can work effectively in practical applications, we can think that the small inaccuracy is because the signal of the real pulsar star we found in this data set is too small compared to the other We can think that the small inaccuracy is due to the fact that in this data set, we find too few real pulsar star signals compared to other signals, which leads to a small error between the predicted data and the actual data used for the test. The way to improve the accuracy is to expand the data set to include enough real pulsar star signals, so as to make our model more accurate.

For the choice of n in our knn model, through figure2 we can find that the accuracy of n starts to increase abruptly at 3, and after 3 it starts to fluctuate slightly and then tends to stabilize gradually, and the highest accuracy is n=19, but we can see from the figure that when n=19 we choose too many interfering signals, and that part of the line graph has already stabilized, so we think that n=3 is the best

### Impact of our findings

Having a highly accurate classifier allows us to predict which data are real pulsar signals and which are interfering signals when we are detecting the signals, so we can save a lot of time in the actual detection and make our detection more efficient.
### Several questions that it could be lead to

1. The pulsar signal is correctly dominant at what the integrated profile excess kurtosis and DM SNR curve STD are equal to, respectively
2. How to make our classifiers more accurate


# References

R. J. Lyon, B. W. Stappers, S. Cooper, J. M. Brooke, J. D. Knowles, Fifty Years of Pulsar Candidate Selection: From simple filters to a new principled real-time classification approach MNRAS, 2016. https://doi.org/10.24432/C5DK6R.

Keith, M.J., Jameson, A., Straten, W.V., Bailes, M., Johnston, S., Kramer, M., Possenti, A., Bates, S., Bhat, N.D., Burgay, M., Burke-Spolaor, S., DAmico, N., Levin, L.A., McMahon, P.L., Milia, S., & Stappers, B.W. (2010). The High Time Resolution Universe Pulsar Survey - I. System configuration and initial discoveries. Journal of Leukocyte Biology.

Dr Robert Lyon, University of Manchester, School of Physics and Astronomy, Alan Turing Building, Manchester M13 9PL, United Kingdom, robert.lyon '@' manchester.ac.uk

Skelly, C. (2017). NASA Continues to Study Pulsars, 50 Years After Their Chance Discovery. NASA. https://www.nasa.gov/feature/goddard/2017/nasa-continues-to-study-pulsars-50-years-after-their-chance-discovery